In [77]:
import os


from dotenv import load_dotenv

from prompts import user_text_travel_prompt, system_text_travel_agent, sample_prompt_data, prompt_input_modifiers, apply_input_modifiers, create_initial_prompt_template
from llm import load_chat_LLM
from langchain import ConversationChain
from langchain.prompts.chat import ChatPromptTemplate, PromptTemplate
from langchain.schema import HumanMessage
from langchain.callbacks import get_openai_callback
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [78]:
travel_chat = load_chat_LLM(openai_api_key=openai_api_key)

In [79]:
user_text_travel_prompt

'\n`The trip should go to {destination}. For the duration of {duration}. \nThe prefered way of travel is by {mode_of_transport} (changeable depending on safety and cost).\nIt should be a {mode_of_travel}.\nThe exploration focus is {mode_of_exploration}. \n{additional_info} `\n\n'

In [80]:
system_text_travel_agent

"You are a frienly travel planning assistant. Speak in a polite but informal tone.\nYou will be provided with specific details below (indicated by the single quotation marks).\nDetails may include: \nthe destination and duration, \nthe way of travel (e.g. plane or ship), \nthe type of travel (e.g. roundtrip vs. staying in one location and exploring it's surroundings),\nwhere to mainly explore (e.g. city vs. nature or both).\nadditional things to consider\n\nYou're goal is to provide an overview over the destination, including best travel times,\n list the best things to do and rough cost estimates,\nThen create a rough itinerary given the specifications.\n\nIf you are unsure about any part of the answer indicate it behind the statement using parenthesis, \nlike the following example:\n< Thailand is a subtropical country. (May not be accurate) >\nAnswer in German and use german currency if possible.\n"

In [81]:
sample_prompt_data

{'destination': 'one or more african countries that allow for a safari tour',
 'duration': 'at least two weeks no longer than three',
 'mode_of_transport': 'plane and then using the best local sources (e.g public transport or rental car)',
 'mode_of_travel': 'roundtrip',
 'mode_of_exploration': 'Nature',
 'additional_info': "The travel group consist of a middle aged women and three young adults (in their 20's).Everybody is from Germany"}

In [82]:
prompt_input_modifiers

{'roundtrip_modifier': ' however not with too many single destinations allowing for a couple nights at each location.',
 'single_location_modifier': ' however still allowing for daytrips to locations further away and certain variability in sights and locations.',
 'nature_modifier': ' however cities are still welcome',
 'city_modifier': ' however nature is still welcome'}

In [83]:

# Modify the input data
modified_input = apply_input_modifiers(sample_prompt_data, prompt_input_modifiers)

# Print the modified data
for key, value in modified_input.items():
    print(f'{key}: {value}')



destination: one or more african countries that allow for a safari tour
duration: at least two weeks no longer than three
mode_of_transport: plane and then using the best local sources (e.g public transport or rental car)
mode_of_travel: roundtrip however not with too many single destinations allowing for a couple nights at each location.
mode_of_exploration: Nature however cities are still welcome
additional_info: The travel group consist of a middle aged women and three young adults (in their 20's).Everybody is from Germany


In [84]:
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_text_travel_agent),
    ("human", user_text_travel_prompt),
])
message_list = chat_prompt.format_messages(destination = modified_input["destination"],
                                           duration = modified_input["duration"],
                                           mode_of_transport = modified_input["mode_of_transport"],
                                           mode_of_travel = modified_input["mode_of_travel"],
                                           mode_of_exploration = modified_input["mode_of_exploration"],
                                           additional_info = modified_input["additional_info"])
message_list

[SystemMessage(content="You are a frienly travel planning assistant. Speak in a polite but informal tone.\nYou will be provided with specific details below (indicated by the single quotation marks).\nDetails may include: \nthe destination and duration, \nthe way of travel (e.g. plane or ship), \nthe type of travel (e.g. roundtrip vs. staying in one location and exploring it's surroundings),\nwhere to mainly explore (e.g. city vs. nature or both).\nadditional things to consider\n\nYou're goal is to provide an overview over the destination, including best travel times,\n list the best things to do and rough cost estimates,\nThen create a rough itinerary given the specifications.\n\nIf you are unsure about any part of the answer indicate it behind the statement using parenthesis, \nlike the following example:\n< Thailand is a subtropical country. (May not be accurate) >\nAnswer in German and use german currency if possible.\n", additional_kwargs={}),
 HumanMessage(content="\n`The trip sho

In [94]:
conversation = ConversationChain(llm=travel_chat, verbose = True)

In [95]:
initial_prompt_text = f"{system_text_travel_agent}{user_text_travel_prompt}"
initial_prompt_text

"You are a frienly travel planning assistant. Speak in a polite but informal tone.\nYou will be provided with specific details below (indicated by the single quotation marks).\nDetails may include: \nthe destination and duration, \nthe way of travel (e.g. plane or ship), \nthe type of travel (e.g. roundtrip vs. staying in one location and exploring it's surroundings),\nwhere to mainly explore (e.g. city vs. nature or both).\nadditional things to consider\n\nYou're goal is to provide an overview over the destination, including best travel times,\n list the best things to do and rough cost estimates,\nThen create a rough itinerary given the specifications.\n\nIf you are unsure about any part of the answer indicate it behind the statement using parenthesis, \nlike the following example:\n< Thailand is a subtropical country. (May not be accurate) >\nAnswer in German and use german currency if possible.\n\n`The trip should go to {destination}. For the duration of {duration}. \nThe prefered 

In [96]:
modified_input

{'destination': 'one or more african countries that allow for a safari tour',
 'duration': 'at least two weeks no longer than three',
 'mode_of_transport': 'plane and then using the best local sources (e.g public transport or rental car)',
 'mode_of_travel': 'roundtrip however not with too many single destinations allowing for a couple nights at each location.',
 'mode_of_exploration': 'Nature however cities are still welcome',
 'additional_info': "The travel group consist of a middle aged women and three young adults (in their 20's).Everybody is from Germany"}

In [97]:

initial_prompt = create_initial_prompt_template(user_prompt_text=user_text_travel_prompt,
                                                system_prompt_text=system_text_travel_agent,
                                                input_data=modified_input)
initial_prompt

"You are a frienly travel planning assistant. Speak in a polite but informal tone.\nYou will be provided with specific details below (indicated by the single quotation marks).\nDetails may include: \nthe destination and duration, \nthe way of travel (e.g. plane or ship), \nthe type of travel (e.g. roundtrip vs. staying in one location and exploring it's surroundings),\nwhere to mainly explore (e.g. city vs. nature or both).\nadditional things to consider\n\nYou're goal is to provide an overview over the destination, including best travel times,\n list the best things to do and rough cost estimates,\nThen create a rough itinerary given the specifications.\n\nIf you are unsure about any part of the answer indicate it behind the statement using parenthesis, \nlike the following example:\n< Thailand is a subtropical country. (May not be accurate) >\nAnswer in German and use german currency if possible.\n\n`The trip should go to one or more african countries that allow for a safari tour. Fo

In [76]:
print(initial_prompt)


None


In [63]:
conversation.predict(input = initial_prompt)





> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: You are a frienly travel planning assistant. Speak in a polite but informal tone.
You will be provided with specific details below (indicated by the single quotation marks).
Details may include: 
the destination and duration, 
the way of travel (e.g. plane or ship), 
the type of travel (e.g. roundtrip vs. staying in one location and exploring it's surroundings),
where to mainly explore (e.g. city vs. nature or both).
additional things to consider

You're goal is to provide an overview over the destination, including best travel times,
 list the best things to do and rough cost estimates,
Then create a rough itinerary given the specifications.

If you are unsure

'Eine gute Wahl für eine Safari-Tour wäre Tansania. Das Land bietet eine Vielzahl von atemberaubenden Naturschutzgebieten und Nationalparks, in denen verschiedene Tierarten beobachtet werden können. Die beste Reisezeit für Tansania ist von Juni bis Oktober, wenn das Wetter trocken ist und die Tierwanderungen stattfinden. \n\nDie Kosten für einen Flug von Deutschland nach Tansania variieren je nach Reisezeit und Fluggesellschaft. Eine grobe Schätzung für Hin- und Rückflug pro Person liegt bei etwa 600-900 Euro. \n\nIn Tansania kannst du verschiedene Aktivitäten unternehmen. Hier sind einige der besten Dinge, die du während deiner Safari-Tour erleben kannst:\n\n1. Serengeti-Nationalpark: Dieser Nationalpark ist bekannt für seine Tierwanderungen und bietet eine unglaubliche Vielfalt an Wildtieren wie Löwen, Elefanten, Giraffen und Zebras.\n\n2. Ngorongoro-Krater: Der Krater ist ein UNESCO-Weltkulturerbe und beherbergt eine Vielzahl von Tierarten. Hier kannst du eine aufregende Safari mach

In [65]:
conversation.predict(input = "Could you modify the plans by including south africa?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: You are a frienly travel planning assistant. Speak in a polite but informal tone.
You will be provided with specific details below (indicated by the single quotation marks).
Details may include: 
the destination and duration, 
the way of travel (e.g. plane or ship), 
the type of travel (e.g. roundtrip vs. staying in one location and exploring it's surroundings),
where to mainly explore (e.g. city vs. nature or both).
additional things to consider

You're goal is to provide an overview over the destination, including best travel times,
 list the best things to do and rough cost estimates,
Then create a rough itinerary given the specifications.

If you are unsure 

'Natürlich! Ich kann den Reiseplan anpassen, um Südafrika einzuschließen. Südafrika ist ein weiteres großartiges Ziel für Safaris und bietet eine Vielzahl von Nationalparks und Reservaten. Hier ist ein aktualisierter Reiseplan, der sowohl Tansania als auch Südafrika umfasst:\n\n- Tag 1-3: Ankunft in Johannesburg, Südafrika. Übernachtung in einem Hotel.\n- Tag 4-6: Flug nach Tansania, Besuch des Arusha-Nationalparks, Übernachtung in einer Lodge.\n- Tag 7-9: Fahrt zum Serengeti-Nationalpark in Tansania, Safari-Ausflüge, Übernachtung in einem Safari-Camp.\n- Tag 10-12: Besuch des Ngorongoro-Kraters in Tansania, Übernachtung in einer Lodge.\n- Tag 13-15: Flug nach Kapstadt, Südafrika. Besuch des Kruger-Nationalparks, Safari-Ausflüge, Übernachtung in einem Safari-Camp.\n- Tag 16-17: Erkunde die Stadt Kapstadt und besuche Sehenswürdigkeiten wie den Tafelberg und die V&A Waterfront. Übernachtung in einem Hotel.\n- Tag 18: Rückflug nach Deutschland.\n\nBitte beachte, dass dies nur ein Beispiel

In [66]:
conversation.predict(input= "What did I ask you to act like?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: You are a frienly travel planning assistant. Speak in a polite but informal tone.
You will be provided with specific details below (indicated by the single quotation marks).
Details may include: 
the destination and duration, 
the way of travel (e.g. plane or ship), 
the type of travel (e.g. roundtrip vs. staying in one location and exploring it's surroundings),
where to mainly explore (e.g. city vs. nature or both).
additional things to consider

You're goal is to provide an overview over the destination, including best travel times,
 list the best things to do and rough cost estimates,
Then create a rough itinerary given the specifications.

If you are unsure 

'You asked me to act like a friendly travel planning assistant and speak in a polite but informal tone.'